In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
columns={'spacegroup' : 'sg',
                            'number_of_total_atoms' : 'Natoms',
                            'percent_atom_al' : 'x_Al',
                            'percent_atom_ga' : 'x_Ga',
                            'percent_atom_in' : 'x_In',
                            'lattice_vector_1_ang' : 'a',
                            'lattice_vector_2_ang' : 'b',
                            'lattice_vector_3_ang' : 'c',
                            'lattice_angle_alpha_degree' : 'alpha',
                            'lattice_angle_beta_degree' : 'beta',
                            'lattice_angle_gamma_degree' : 'gamma',
                            'formation_energy_ev_natom' : 'E',
                            'bandgap_energy_ev' : 'Eg'}
    
    
df_train = pd.read_csv("./input/train.csv").rename(columns=columns)
df_train["dataset"] = "train"
df_train["E"]=np.log1p(df_train["E"])
df_train["Eg"]=np.log1p(df_train["Eg"])
df_test = pd.read_csv("./input/test.csv").rename(columns=columns)
df_test["dataset"] = "test"
df_total = pd.concat([df_train, df_test], ignore_index=True)

len(df_train),len(df_test),len(df_total)

(2400, 600, 3000)

In [3]:
df_total.head()

,E,Eg,Natoms,a,alpha,b,beta,c,dataset,gamma,id,sg,x_Al,x_Ga,x_In
0,0.065788,1.490362,80.0,9.9523,90.0026,8.5513,90.0023,9.1775,train,90.0017,1,33,0.6250,0.3750,0.000
1,0.222343,1.366347,80.0,6.1840,90.0186,6.1838,89.9980,23.6287,train,120.0025,2,194,0.6250,0.3750,0.000
2,0.167293,1.320101,40.0,9.7510,90.9688,5.6595,91.1228,13.9630,train,30.5185,3,227,0.8125,0.1875,0.000
3,0.196553,1.469992,30.0,5.0036,89.9888,5.0034,90.0119,13.5318,train,120.0017,4,167,0.7500,0.0000,0.250
4,0.049266,0.866806,80.0,6.6614,89.9960,6.6612,90.0006,24.5813,train,119.9893,5,194,0.0000,0.6250,0.375


In [4]:
df_total.tail()

,E,Eg,Natoms,a,alpha,b,beta,c,dataset,gamma,id,sg,x_Al,x_Ga,x_In
2995,NaN,NaN,80.0,24.8145,90.0002,6.3964,104.7733,6.2933,test,90.0001,596,12,0.000,0.5938,0.4062
2996,NaN,NaN,40.0,5.5783,90.0008,9.4849,89.9967,10.1107,test,90.0004,597,33,0.125,0.0000,0.8750
2997,NaN,NaN,80.0,6.9377,90.0072,6.9372,89.9880,25.0641,test,119.9857,598,194,0.000,0.2500,0.7500
2998,NaN,NaN,40.0,5.1841,90.0041,8.8659,90.0009,9.4956,test,90.0007,599,33,0.625,0.0000,0.3750
2999,NaN,NaN,80.0,9.4959,90.0029,9.4956,90.0031,9.4956,test,89.9969,600,206,0.375,0.3438,0.2812


In [5]:
#from https://www.kaggle.com/cbartel/random-forest-using-elemental-properties
def get_vol(a, b, c, alpha, beta, gamma):
    """
    Args:
        a (float) - lattice vector 1
        b (float) - lattice vector 2
        c (float) - lattice vector 3
        alpha (float) - lattice angle 1 [radians]
        beta (float) - lattice angle 2 [radians]
        gamma (float) - lattice angle 3 [radians]
    Returns:
        volume (float) of the parallelepiped unit cell
    """
    alpha=alpha*np.pi/180
    beta=beta*np.pi/180
    gamma=gamma*np.pi/180
    return a*b*c*np.sqrt(1 + 2*np.cos(alpha)*np.cos(beta)*np.cos(gamma)
                           - np.cos(alpha)**2
                           - np.cos(beta)**2
                           - np.cos(gamma)**2)


    
# compute the cell volumes 
df_total['vol'] = get_vol(df_total['a'], df_total['b'], df_total['c'],
                          df_total['alpha'], df_total['beta'], df_total['gamma'])
#df_total[['a','b','c','alpha','beta','gamma','vol']].head()
df_total['density']=df_total['Natoms']/df_total["vol"]
df_total['density_Al']=df_total['density']*df_total['x_Al']
df_total['density_Ga']=df_total['density']*df_total['x_Ga']
df_total['density_In']=df_total['density']*df_total['x_In']
df_total['sg']=df_total['sg'].astype('category')

In [6]:
df_total.head()

,E,Eg,Natoms,a,alpha,b,beta,c,dataset,gamma,id,sg,x_Al,x_Ga,x_In,vol,density,density_Al,density_Ga,density_In
0,0.065788,1.490362,80.0,9.9523,90.0026,8.5513,90.0023,9.1775,train,90.0017,1,33,0.6250,0.3750,0.000,781.052081,0.102426,0.064016,0.038410,0.000000
1,0.222343,1.366347,80.0,6.1840,90.0186,6.1838,89.9980,23.6287,train,120.0025,2,194,0.6250,0.3750,0.000,782.500110,0.102236,0.063898,0.038339,0.000000
2,0.167293,1.320101,40.0,9.7510,90.9688,5.6595,91.1228,13.9630,train,30.5185,3,227,0.8125,0.1875,0.000,391.227531,0.102242,0.083072,0.019170,0.000000
3,0.196553,1.469992,30.0,5.0036,89.9888,5.0034,90.0119,13.5318,train,120.0017,4,167,0.7500,0.0000,0.250,293.377334,0.102257,0.076693,0.000000,0.025564
4,0.049266,0.866806,80.0,6.6614,89.9960,6.6612,90.0006,24.5813,train,119.9893,5,194,0.0000,0.6250,0.375,944.713843,0.084682,0.000000,0.052926,0.031756


In [7]:
#Encoding of cat features
import sys 
sys.path.append("../kaggle_varie")
from  varie import *
cols_to_enc=["sg"]

#binary encoder
#enc=bin_enc(df_total,cols_to_enc,verbose=2,copy=True,drop_original=True,ordinal_only=False)
#one-hot encoder
enc=pd.get_dummies(df_total,columns=cols_to_enc)

/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions a

In [95]:
def grid_search_fct(model,params,df,y_col,n_iter=20,cv=4,drop_col=[],verbose=2):
    
    X_train=df.drop(y_col+drop_col,axis=1).values
    grids=[]
    for y in y_col:
        print(y)
        y_train=df[y].values
        print(X_train.shape,y_train.shape)

        grid=RandomizedSearchCV(model,param_distributions=params, n_iter=n_iter,cv=cv,verbose=verbose,scoring="neg_mean_squared_error" )

        grid.fit(X_train,y_train)
        grids.append(grid)
    return grids

In [121]:
MLPRegressor?

In [137]:
AdaBoostRegressor?

In [140]:
#grid search for random forest
import scipy
from  sklearn.model_selection import RandomizedSearchCV
from sklearn import *
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.ensemble import  GradientBoostingRegressor, RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
#from sklearn.kernel_approximation import Nystroem
#from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import Lasso,Ridge
from xgboost import XGBRegressor
#from lightgbm import LGBMRegressor


def lognuniform(low=0, high=1, size=None, base=10):
    return np.power(base, np.random.uniform(low, high, size))

y_col=["E","Eg"]
drop_col=["id","dataset"]
df_total_train_eval=enc[df_total.dataset=='train']
df_total_test=enc[df_total.dataset=='test']

X_train=df_total_train_eval.drop(y_col+drop_col,axis=1).values
X_test=df_total_test.drop(y_col+drop_col,axis=1).values

models={
    
#    'knn':
 #          (KNeighborsRegressor(),
  #          {'n_neighbors':scipy.stats.randint(1,100)}),
    
#    'svr':
 #          (SVR(verbose=False),
  #          {'C':lognuniform(low=-4,high=4,base=10,size=100),
   #             'epsilon':lognuniform(low=-2,high=0,base=10,size=100)}),

#    'rf':
#           (ensemble.RandomForestRegressor(verbose=False),
#            {"max_depth": scipy.stats.randint(1,100), 
#             'n_estimators': scipy.stats.randint(1,400),
#             'max_features':('log2','sqrt','auto')}),
    
#    'cb':
#           (CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE',verbose=False),
#            {"depth": scipy.stats.randint(1,5), 
#             'iterations': scipy.stats.randint(1000,2000),
#             'learning_rate':lognuniform(low=-2,high=-1,base=10,size=100)}),
    
#    'mlp': 
#           (MLPRegressor((80, 10), early_stopping=False),
#             {'hidden_layer_sizes':scipy.stats.randint(1,100),
#              'alpha':lognuniform(low=-5,high=-1,base=10,size=100)}),
             
 #    'gb':
 #          (GradientBoostingRegressor(n_estimators=100),
 #           {'learning_rate':lognuniform(low=-3,high=-1,base=10,size=100), 
 #            'n_estimators': scipy.stats.randint(1,300)}),
    
#    'lasso':
#            (Lasso(),
#            {'alpha':lognuniform(low=-4,high=4,base=10,size=100)}),  

#    'ridge':
#            (Ridge(),
#            {'alpha':lognuniform(low=-4,high=4,base=10,size=100)}),
    
 #   'xgb':
  #      (XGBRegressor(),
   #      {'max_depth':scipy.stats.randint(1,100), 
    #      'learning_rate':lognuniform(low=-4,high=-0.5,base=10,size=100), 
     #     'n_estimators':scipy.stats.randint(1,400)}),
    
 #does not install    
 #   'gbm' :
 #       (lgb.LGBMRegressor(objective='regression'),
 #           {'num_leaves':scipy.stats.randint(1,200), 
 #         'learning_rate':lognuniform(low=-4,high=-0.5,base=10,size=100), 
 #         'n_estimators':scipy.stats.randint(1,400)}),
    'adb' :
        (AdaBoostRegressor(loss="square"),
            {'learning_rate':lognuniform(low=-4,high=-0.1,base=10,size=100), 
             'n_estimators':scipy.stats.randint(1,400)}),         

    
       }
 
    

#results={}
for (tag,model) in  models.items():
    print(tag)
    results[tag]=grid_search_fct(model[0],model[1],df_total_train_eval,y_col,n_iter=20,cv=4,drop_col=drop_col)


    
    #grid=RandomizedSearchCV(model[0],param_distributions=params, n_iter=20,cv=4,verbose=2,scoring="neg_mean_squared_error" )

                        
    #grid.fit(X_train,y_train)
    #grids.append(grid)

adb
E
(2400, 21) (2400,)
Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] n_estimators=235, learning_rate=0.07109792420131451 .............
[CV]  n_estimators=235, learning_rate=0.07109792420131451, total=   0.4s
[CV] n_estimators=235, learning_rate=0.07109792420131451 .............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV]  n_estimators=235, learning_rate=0.07109792420131451, total=   0.5s
[CV] n_estimators=235, learning_rate=0.07109792420131451 .............
[CV]  n_estimators=235, learning_rate=0.07109792420131451, total=   0.8s
[CV] n_estimators=235, learning_rate=0.07109792420131451 .............
[CV]  n_estimators=235, learning_rate=0.07109792420131451, total=   0.8s
[CV] n_estimators=153, learning_rate=0.0022317174041792715 ...........
[CV]  n_estimators=153, learning_rate=0.0022317174041792715, total=   0.7s
[CV] n_estimators=153, learning_rate=0.0022317174041792715 ...........
[CV]  n_estimators=153, learning_rate=0.0022317174041792715, total=   0.7s
[CV] n_estimators=153, learning_rate=0.0022317174041792715 ...........
[CV]  n_estimators=153, learning_rate=0.0022317174041792715, total=   0.7s
[CV] n_estimators=153, learning_rate=0.0022317174041792715 ...........
[CV]  n_estimators=153, learning_rate=0.0022317174041792715, total=   0.7s
[CV] n_estimators=366, learning_rate=0.0024020430925079

[CV]  n_estimators=153, learning_rate=0.004330186866161943, total=   0.7s
[CV] n_estimators=153, learning_rate=0.004330186866161943 ............
[CV]  n_estimators=153, learning_rate=0.004330186866161943, total=   0.7s
[CV] n_estimators=294, learning_rate=0.003071022281488235 ............
[CV]  n_estimators=294, learning_rate=0.003071022281488235, total=   1.3s
[CV] n_estimators=294, learning_rate=0.003071022281488235 ............
[CV]  n_estimators=294, learning_rate=0.003071022281488235, total=   1.4s
[CV] n_estimators=294, learning_rate=0.003071022281488235 ............
[CV]  n_estimators=294, learning_rate=0.003071022281488235, total=   1.3s
[CV] n_estimators=294, learning_rate=0.003071022281488235 ............
[CV]  n_estimators=294, learning_rate=0.003071022281488235, total=   1.4s
[CV] n_estimators=323, learning_rate=0.5586995159146345 ..............
[CV]  n_estimators=323, learning_rate=0.5586995159146345, total=   0.1s
[CV] n_estimators=323, learning_rate=0.5586995159146345 ..

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  1.0min finished


Eg
(2400, 21) (2400,)
Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] n_estimators=298, learning_rate=0.013751899574428378 ............
[CV]  n_estimators=298, learning_rate=0.013751899574428378, total=   1.3s
[CV] n_estimators=298, learning_rate=0.013751899574428378 ............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV]  n_estimators=298, learning_rate=0.013751899574428378, total=   1.3s
[CV] n_estimators=298, learning_rate=0.013751899574428378 ............
[CV]  n_estimators=298, learning_rate=0.013751899574428378, total=   1.3s
[CV] n_estimators=298, learning_rate=0.013751899574428378 ............
[CV]  n_estimators=298, learning_rate=0.013751899574428378, total=   1.3s
[CV] n_estimators=114, learning_rate=0.040475148528734475 ............
[CV]  n_estimators=114, learning_rate=0.040475148528734475, total=   0.5s
[CV] n_estimators=114, learning_rate=0.040475148528734475 ............
[CV]  n_estimators=114, learning_rate=0.040475148528734475, total=   0.5s
[CV] n_estimators=114, learning_rate=0.040475148528734475 ............
[CV]  n_estimators=114, learning_rate=0.040475148528734475, total=   0.5s
[CV] n_estimators=114, learning_rate=0.040475148528734475 ............
[CV]  n_estimators=114, learning_rate=0.040475148528734475, total=   0.5s
[CV] n_estimators=80, learning_rate=0.7410505900629079 .

[CV]  n_estimators=387, learning_rate=0.009723521346457912, total=   1.7s
[CV] n_estimators=387, learning_rate=0.009723521346457912 ............
[CV]  n_estimators=387, learning_rate=0.009723521346457912, total=   1.7s
[CV] n_estimators=355, learning_rate=0.001015141334160562 ............
[CV]  n_estimators=355, learning_rate=0.001015141334160562, total=   1.6s
[CV] n_estimators=355, learning_rate=0.001015141334160562 ............
[CV]  n_estimators=355, learning_rate=0.001015141334160562, total=   1.6s
[CV] n_estimators=355, learning_rate=0.001015141334160562 ............
[CV]  n_estimators=355, learning_rate=0.001015141334160562, total=   1.7s
[CV] n_estimators=355, learning_rate=0.001015141334160562 ............
[CV]  n_estimators=355, learning_rate=0.001015141334160562, total=   1.7s
[CV] n_estimators=259, learning_rate=0.0024020430925079488 ...........
[CV]  n_estimators=259, learning_rate=0.0024020430925079488, total=   1.2s
[CV] n_estimators=259, learning_rate=0.0024020430925079

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  1.1min finished


In [141]:
#best model and its performance

for tag,grids in results.items():
    print(tag)
    for grid in grids:
        print(grid.best_params_)
    print((np.sqrt(-grids[0].best_score_)+np.sqrt(-grids[1].best_score_))/2)

lasso
{'alpha': 0.0027387754984453234}
{'alpha': 0.00016338531868553138}
0.08074507599068162
adb
{'n_estimators': 235, 'learning_rate': 0.07109792420131451}
{'n_estimators': 114, 'learning_rate': 0.040475148528734475}
0.0796619233301889
gb
{'n_estimators': 276, 'learning_rate': 0.026887651632063816}
{'n_estimators': 136, 'learning_rate': 0.08546898120651171}
0.06106657593199877
knn
{'n_neighbors': 8}
{'n_neighbors': 8}
0.08431301064116939
rf
{'max_depth': 11, 'n_estimators': 98, 'max_features': 'sqrt'}
{'max_depth': 9, 'n_estimators': 190, 'max_features': 'log2'}
0.06283712300468994
xgb
{'max_depth': 45, 'n_estimators': 85, 'learning_rate': 0.06559377246367769}
{'max_depth': 26, 'n_estimators': 149, 'learning_rate': 0.027448151770068534}
0.06903262121078503
svr
{'C': 3.6229030858475677, 'epsilon': 0.03911398169420586}
{'C': 2.296995560428917, 'epsilon': 0.0164353456239699}
0.09618054076145428
cb
{'iterations': 1734, 'learning_rate': 0.016556953835177485, 'depth': 4}
{'iterations': 1679

In [142]:
learners1=[grid[0].best_estimator_ for grid in results.values()]
learners2=[grid[1].best_estimator_ for grid in results.values()]

## Ensembles

In [144]:
from mlens.ensemble import SuperLearner
import mlens
from mlens.model_selection import Evaluator
from mlens.metrics import make_scorer
from mlens.metrics import rmse

# Instantiate the ensemble with 10 folds
meta_learner=CatBoostRegressor(iterations=1200,
                            learning_rate=0.03,
                            depth=4,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=SEED,
                            od_type='Iter',
                            od_wait=50,verbose=False)

sl1 = SuperLearner(
    folds=5,
    verbose=True,
    random_state=SEED,
    scorer=mlens.metrics.rmse
)
sl2 = SuperLearner(
    folds=5,
    verbose=True,
    random_state=SEED,
    scorer=mlens.metrics.rmse
)

# Add the base learners and the meta learner
sl1.add(learners1) 
sl1.add_meta(meta_learner)
sl2.add(learners2) 
sl2.add_meta(meta_learner)


sls=[sl1,sl2]
#evaluator
#evl = Evaluator(make_scorer(mlens.metrics.rmse), cv=5, shuffle=False)

for i,y in enumerate(y_col):
    print(y)
    y_train=df_total_train_eval[y].values
    y_test=df_total_test[y].values
    print(X_train.shape,y_train.shape)
    
    #evl.fit(X_train, y_train, sl, {}, n_iter=1)

    # Train the ensemble
    sls[i].fit(X_train, y_train)
    preds = sls[i].predict(X_train)
    print(rmse(y_train, preds))
#    results.append(mlens.metrics.rmse(y_train, ensemble.predict(X_train)),
#                          evl.summary['test_score_mean']['superlearner'],
#                          evl.summary['test_score_std']['superlearner'],
#                          mlens.metrics.rmse(y_test, ensemble.predict(X_test)))

#    print_scores(scores_df, 'mlens')

E
(2400, 21) (2400,)

Fitting 2 layers


/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/mlens/externals/sklearn/base.py:116: DeprecationWarning: Estimator CatBoostRegressor modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)
/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/mlens/externals/sklearn/base.py:116: DeprecationWarning: Estimator CatBoostRegressor modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  % type(estimator).__name__, DeprecationWarning)


Fit complete                        | 00:00:48

Predicting 2 layers
Predict complete                    | 00:00:00
0.028519001794062512
Eg
(2400, 21) (2400,)

Fitting 2 layers


/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Fit complete                        | 00:00:39

Predicting 2 layers
Predict complete                    | 00:00:00
0.07582528911912395


In [145]:
#write to csv
%load_ext autoreload
%aimport varie
%autoreload 2
#I use a different model for E and Eg
varie.make_csv2(df_total_train_eval,pd.DataFrame(),df_total_test,
#         (ensemble.RandomForestRegressor(max_depth= 11, max_features='log2', n_estimators= 55),
#          ensemble.RandomForestRegressor(max_depth= 9, max_features='sqrt', n_estimators= 220)),
            (sl1,sl2),
         y_col,'sl2.csv',drop=drop_col,columns=['id','E','Eg'],
         new_column_names=['id','formation_energy_ev_natom' ,'bandgap_energy_ev'],change_col_names=True,fit=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
E SuperLearner(array_check=2, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=235, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=5, raise_on_exc...b199d8>)],
   n_jobs=-1, name='group-33', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=1, sample_size=20,
       scorer=<function rmse at 0x7fd5c5b199d8>, shuffle=False,
       verbose=True)
shapes: (2400, 21) (2400,)

Predicting 2 layers
Predict complete                    | 00:00:00
Eg SuperLearner(array_check=2, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', pro

../kaggle_varie/varie.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_test[y_]=y_pred


Predict complete                    | 00:00:00


../kaggle_varie/varie.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_test[y_]=y_pred


In [ ]:
#ensemble = SuperLearner(folds=5, scorer=mse)
#ensemble.add([xgb.XGBRegressor(), lgb.LGBMRegressor(n_estimators=30)])
#ensemble.add_meta([LinearRegression()])

evl = Evaluator(make_scorer(mlens.metrics.rmse), cv=5, shuffle=False)
evl.fit(X_train, y_train, sl, {}, n_iter=1)


scores_df['mlens'] = [mlens.metrics.rmse(y_train, ensemble.predict(X_train)),
                      evl.summary['test_score_mean']['superlearner'],
                      evl.summary['test_score_std']['superlearner'],
                      mlens.metrics.rmse(y_test, ensemble.predict(X_test))]

print_scores(scores_df, 'mlens')

In [ ]:
evaluator = Evaluator()
evaluator.fit(X_train, y_train,sl)